# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
def run_query(query, type_query, row=()):
    """Run a query in Apache Cassandra.
    
    Parameters:        
       query (string): the string of the query. 
       type_query (string): the type of query: select, create or drop.
       row (tuple): a tuple to be inserted. 
    
    Returns: 
        None if the function executes a select or drop operation, 
        a result set if the function executes  .
    """ 
    
    if type_query == "insert":
        try:
            rows = session.execute(query, row)
            return None
        except Exception as e:
            print(e)        
        
    elif type_query == "select":
        try:
            rows = session.execute(query)
            return rows 
        except Exception as e:
            print(e)
    
    else:
        try:
            session.execute(query)
            return None 
        except Exception as e:   
            print(e)        
        

#### Creating list of filepaths to process original event csv data files

In [3]:
def process_filepaths():
    """Create the list of filepaths to process original 
       event csv data files.
    
    Parameters:        
       Nothing, this function does not have parameters. 
    
    Returns: 
        file_path_list (list): a list of filepaths.
    """ 
    
    
    # Get your current folder and subfolder event data
    filepath = os.getcwd() + '/event_data'

    # Create a for loop to create a list of files and collect each filepath
    for root, dirs, files in os.walk(filepath):
        # join the file path and roots with the subdirectories using glob
        file_path_list = glob.glob(os.path.join(root,'*'))
        
    return file_path_list        

In [4]:
file_path_list = process_filepaths()

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
def process_files(file_path_list, output_file):
    """Process the files to create the data file csc that will be used 
       for Apacha Cassandra tables.
    
    Parameters:        
       file_path_list (list): a list of filepaths. 
       output_file (string): the name of the csv file that will be used for
                             Apache Cassandra.
    
    Returns: 
        None, the function simply processes files and creates a csv file.
    """ 
    
    # initiating an empty list of rows that will be generated from each file
    full_data_rows_list = [] 

    
    # for every filepath in the file path list 
    for f in file_path_list:
        # reading csv file 
        with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
            # creating a csv reader object 
            csvreader = csv.reader(csvfile) 
            next(csvreader)
        
            # extracting each data row one by one and append it        
            for line in csvreader:
                full_data_rows_list.append(line) 
            
    # creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
    # Apache Cassandra tables
    csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

    with open(output_file, 'w', encoding = 'utf8', newline='') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
        for row in full_data_rows_list:
            if (row[0] == ''):
                continue
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
        

In [6]:
process_files(file_path_list, 'event_datafile_new.csv')

In [7]:
! head event_datafile_new.csv

"artist","firstName","gender","itemInSession","lastName","length","level","location","sessionId","song","userId"
"Stephen Lynch","Jayden","M","0","Bell","182.85669","free","Dallas-Fort Worth-Arlington, TX","829","Jim Henson's Dead","91"
"Manowar","Jacob","M","0","Klein","247.562","paid","Tampa-St. Petersburg-Clearwater, FL","1049","Shell Shock","73"
"Morcheeba","Jacob","M","1","Klein","257.41016","paid","Tampa-St. Petersburg-Clearwater, FL","1049","Women Lose Weight (Feat: Slick Rick)","73"
"Maroon 5","Jacob","M","2","Klein","231.23546","paid","Tampa-St. Petersburg-Clearwater, FL","1049","Won't Go Home Without You","73"
"Train","Jacob","M","3","Klein","216.76363","paid","Tampa-St. Petersburg-Clearwater, FL","1049","Hey_ Soul Sister","73"
"LMFAO","Jacob","M","4","Klein","227.99628","paid","Tampa-St. Petersburg-Clearwater, FL","1049","I'm In Miami Bitch","73"
"DJ Dizzy","Jacob","M","5","Klein","221.1522","paid","Tampa-St. Petersburg-Clearwater, FL","1049","Sexy Bitch","73"
"Fish Go Deep 

In [8]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding section. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [9]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [10]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1} """
)
    
except Exception as e:
    print(e)

#### Set Keyspace

In [11]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Task 1: Getting the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4.

For this query itemInSession and sessionId are considered as the composite key, because this query was thought to recover values for sessionId and itemInSession. SessionId is the partition key and its role is to spread data evenly around the cluster. ItemInSession is a cluster column which allows to sort the data. From the above, we meet the requirements of the query. 

In [12]:
query1 = "CREATE TABLE IF NOT EXISTS song_information "
query1 = query1 + " (session_id int, item_in_session int, \
                     artist text, \
                     song_title text, \
                     song_length float, \
                     PRIMARY KEY (session_id, item_in_session))"

run_query(query1, "create")

In [13]:
file = 'event_datafile_new.csv'


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_information (session_id, item_in_session, \
                 artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        run_query(query, "insert", (int(line[8]), int(line[3]), line[0], line[9],float(line[5])) )        
        

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
query = "SELECT artist, song_title, song_length FROM song_information "
query = query + "WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = run_query(query, "select")
        
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: {}, Song Title: {}, Song Length: {}".format
          (row.artist, row.song_title, row.song_length))

Artist: Faithless, Song Title: Music Matters (Mark Knight Dub), Song Length: 495.30731201171875


## Task 2: Getting the the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.

For this query userId, sessionId, itemInSession  are considered as the composite key. UserId and SessionId are a composite partition key and its role is to spread data evenly around the cluster. ItemInSession is a cluster column which allows to sort the data. From the above, we meet the requirements of the query. 

In [15]:
query2 = "CREATE TABLE IF NOT EXISTS activity_information "
query2 = query2 + " (user_id int, session_id int, item_in_session int, \
                     artist text, \
                     song_title text, \
                     first_name_user text, \
                     last_name_user text, \
                     PRIMARY KEY ((user_id, session_id), item_in_session))"

run_query(query2, "create")

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO activity_information (user_id, \
                 session_id, item_in_session, \
                 artist, song_title, \
                 first_name_user, last_name_user \
                 )"        
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        run_query(query, "insert", (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]) )


In [17]:
query = "SELECT artist, song_title, first_name_user, last_name_user \
         FROM activity_information "
query = query + "WHERE user_id = 10 AND session_id = 182"
try:
    rows = run_query(query, "select")
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: {}, Song Title: {}, \
           first_name_user: {}, last_name_user: {}".format(row.artist, 
           row.song_title, row.first_name_user, row.last_name_user))

Artist: Down To The Bone, Song Title: Keep On Keepin' On,            first_name_user: Sylvie, last_name_user: Cruz
Artist: Three Drives, Song Title: Greece 2000,            first_name_user: Sylvie, last_name_user: Cruz
Artist: Sebastien Tellier, Song Title: Kilometer,            first_name_user: Sylvie, last_name_user: Cruz
Artist: Lonnie Gordon, Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit),            first_name_user: Sylvie, last_name_user: Cruz


## Task3: Getting every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

For this query songTitle and userId are considered as the composite key. SongTitle is the partition key and its role is to spread data evenly around the cluster. UserId is a cluster column which allows to sort the data. We have to include userId since the combination of first and last name could not be unique. We suppose that the userId is greater than 0. From the above, we meet the requirements of the query. 

In [18]:
query3 = "CREATE TABLE IF NOT EXISTS user_song_information "
query3 = query3 + " (song_title text, \
                     user_id int, \
                     first_name_user text, \
                     last_name_user text, \
                     PRIMARY KEY (song_title, user_id))"

try:
    run_query(query3, "create")
except Exception as e:
    print(e)        

Cassandra is optimized for writes that is why we do not care about if we will insert the same item, cassandra simply will replace it 

https://medium.com/coinmonks/cassandra-distributed-key-value-store-optimized-for-write-heavy-workloads-77f69c01388c

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song_information (song_title, user_id, \
                 first_name_user, \
                 last_name_user)"
        query = query + "VALUES (%s, %s, %s, %s)"
        run_query(query, "insert", (line[9], int(line[10]), line[1], line[4]))

In [20]:
query = "SELECT first_name_user, last_name_user \
         FROM user_song_information "
query = query + "WHERE song_title = 'All Hands Against His Own' AND user_id > 0"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print("first_name_user: {}, last_name_user: {}".format
          (row.first_name_user, row.last_name_user))

first_name_user: Jacqueline, last_name_user: Lynch
first_name_user: Tegan, last_name_user: Levine
first_name_user: Sara, last_name_user: Johnson


### Drop the tables before closing out the sessions

In [21]:
query = "drop table song_information"
run_query(query, "drop")

query = "drop table activity_information"
run_query(query, "drop")
    
query = "drop table user_song_information"
run_query(query, "drop")

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()